In [13]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, BatchNormalization, Embedding
from keras.layers.core import Dense, Activation, Dropout
from keras.utils import np_utils, pad_sequences
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go 
import plotly.express as px
import plotly.figure_factory as ff

In [3]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  print('The TPU Cluster Resolver is '+ tpu.master())
except ValueError:
  print("The Value Error has occurred")
  tpu = None

if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
  strategy = tf.distribute.get_strategy()

print(f'The number of replicas are {strategy.num_replicas_in_sync}')

The TPU Cluster Resolver is grpc://10.46.152.138:8470


The number of replicas are 8


In [4]:
train = pd.read_csv('/content/drive/MyDrive/Datasets/Zero To Transformers Series/jigsaw-toxic-comment-train.csv')
test = pd.read_csv('/content/drive/MyDrive/Datasets/Zero To Transformers Series/test.csv')
validation = pd.read_csv('/content/drive/MyDrive/Datasets/Zero To Transformers Series/validation.csv')

In [5]:
train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [6]:
train.drop(['severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)

In [7]:
train.shape

(223549, 3)

In [8]:
train = train.loc[:12000,:]
train.shape

(12001, 3)

In [9]:
train['comment_text'].apply(lambda x: len(str(x).split())).max()

1403

In [10]:
def roc_auc(prediction, targets):
  tpr, fpr, threshold = metrics.roc_curve(targets, prediction)
  roc_auc = metrics.auc(tpr, fpr)
  return roc_auc

In [11]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values, 
                                                  stratify=train.toxic.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

Recurrent Neural Networks

In [14]:
token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [15]:
%%time
with strategy.scope():
  model = Sequential()
  model.add(Embedding(len(word_index)+1, 300, input_length=max_len))
  model.add(SimpleRNN(100))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1500, 300)         13049100  
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               40100     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 13,089,301
Trainable params: 13,089,301
Non-trainable params: 0
_________________________________________________________________
CPU times: user 668 ms, sys: 50 ms, total: 718 ms
Wall time: 2.52 s


In [18]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
19/19 [==============================] - 16s 335ms/step - loss: 0.3653 - accuracy: 0.8792
Epoch 2/5
19/19 [==============================] - 2s 86ms/step - loss: 0.2961 - accuracy: 0.9053
Epoch 3/5
19/19 [==============================] - 2s 86ms/step - loss: 0.2397 - accuracy: 0.9087
Epoch 4/5
19/19 [==============================] - 2s 86ms/step - loss: 0.1583 - accuracy: 0.9464
Epoch 5/5
19/19 [==============================] - 2s 86ms/step - loss: 0.0724 - accuracy: 0.9781


In [25]:
scores = model.predict(xvalid_pad)
print(f'The roc scores are {roc_auc(scores, yvalid): .2f}')

76/76 [==============================] - 2s 19ms/step
The roc scores are  0.79


In [26]:
scores_model = []
scores_model.append({'Model': 'SimpleRNN', 'AUC Score': roc_auc(scores, yvalid)})

Word Embeddings

In [27]:
#from google.colab import files

#uploaded = files.upload()

#for fn in uploaded.keys():
#  print('User uploaded file "{name}" with length {length} bytes'.format(
#      name=fn, length=len(uploaded[fn])))

KeyboardInterrupt: ignored

In [33]:
!wget http://www-nlp.stanford.edu/data/glove.840B.300d.zip

--2023-06-10 03:05:13--  http://www-nlp.stanford.edu/data/glove.840B.300d.zip
Resolving www-nlp.stanford.edu (www-nlp.stanford.edu)... 171.64.67.140
Connecting to www-nlp.stanford.edu (www-nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2023-06-10 03:05:13--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2023-06-10 03:05:13--  https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTT

In [34]:
!unzip glove.840B.300d.zip

Archive:  glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


In [35]:
!ls
!pwd

drive  glove.840B.300d.txt  glove.840B.300d.zip  sample_data
/content


In [37]:
embeddings_index = {}
f = open('/content/glove.840B.300d.txt', 'r', encoding='utf-8')
for line in tqdm(f):
  values = line.split(' ')
  key = values[0]
  coeff = np.asarray([float(val) for val in values[1:]])
  embeddings_index[key] = coeff
f.close()

2196017it [03:59, 9160.56it/s] 


In [38]:
print(f'Found {len(embeddings_index)} word vectors')

Found 2196016 word vectors


Long Short Term Memory

In [39]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index)+1, 300))
for word, i in tqdm(word_index.items()):
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

100%|██████████| 43496/43496 [00:00<00:00, 65661.75it/s]


In [40]:
%%time
with strategy.scope():
  model = Sequential()
  model.add(Embedding(len(word_index)+1, 300, weights=[embedding_matrix], input_length=max_len, trainable=False ))
  model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1500, 300)         13049100  
                                                                 
 lstm (LSTM)                 (None, 100)               160400    
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 13,209,601
Trainable params: 160,501
Non-trainable params: 13,049,100
_________________________________________________________________
CPU times: user 972 ms, sys: 1.27 s, total: 2.24 s
Wall time: 5.6 s


In [42]:
model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
19/19 [==============================] - 14s 345ms/step - loss: 0.3812 - accuracy: 0.8564
Epoch 2/5
19/19 [==============================] - 4s 194ms/step - loss: 0.2470 - accuracy: 0.9156
Epoch 3/5
19/19 [==============================] - 4s 194ms/step - loss: 0.1833 - accuracy: 0.9320
Epoch 4/5
19/19 [==============================] - 4s 199ms/step - loss: 0.1508 - accuracy: 0.9436
Epoch 5/5
19/19 [==============================] - 4s 193ms/step - loss: 0.1392 - accuracy: 0.9470


In [43]:
scores = model.predict(xvalid_pad)

76/76 [==============================] - 6s 62ms/step


In [44]:
print(f'The roc_score is {(roc_auc(scores, yvalid)): .2f}')

The roc_score is  0.97


In [45]:
scores_model.append({'model':'LSTM', 'ROC Score': roc_auc(scores, yvalid)})

In [46]:
scores_model

[{'Model': 'SimpleRNN', 'AUC Score': 0.7917357314517992},
 {'model': 'LSTM', 'ROC Score': 0.972648318736854}]